# Network Latency 

AWS services and readymade solutions make user's life easy if one knows how to use them. 
These centers are distributed globally in a strategic way to offer best possible service to customers. 
Geographic location of physical servers plays important role in the response time of the services. 
A cloud user in Australia will prefer his server to be in sydney data center rather than virginia data center.
Users have the flexibility and liberty to choose where they want their physical machines to be located.  
Below image shows all locations of aws data centers and service regions.

<img src="../images/AWS_infrastructure.PNG">

Leran more about these regions [here](https://aws.amazon.com/about-aws/global-infrastructure/)

In the [Launch_Apache_Server](Launch_Apache_Server.ipynb) lab, we have set up a web server and displayed a static webpage with the instance information. In this lab, we will look into some of the basic networking aspects. 


* So what is the significance of geographical location of servers. 
* Will it affect the usage and service levels. 
* Will it affect the quality if service users experience. 

Lets do a simple demonstration by pinging servers in same availability zones and ones elsewhere.  

### What is network latency?

Network latency indicates delays that happen in data communication over a network. 
If the delays are small, 
then connection is called a low-latency network,
but if the network connection is choked with long delays then they are called high latency networks.


Normally network latency is seen often in live transmissions/streaming. 
The communication hops between a ground transmitter, 
a satellite and from a satellite to a receiver each take time. 
This latency is the wait time introduced by the signal travelling the geographical distance as well as over the various pieces of communications equipment.

We will have three ec2 instances created, two in Oregon data center and one in Dublin, Ireland. 
We will compare the response times to ping servers in same data center vs pin serves in different data centers. 


**Create EC2 client to launch instances**

In [ ]:
################################### SET THE FOLLOWING PARAMETERS ###################################################
#Set the AWS Region
region = 'us-west-2'


ami_image = 'ami-e689729e'

#Set the AWS Access ID (Given to you buy the DSA staff)
access_id = 'Put your Access ID here'

#Set the AWS Access Key (Given to you buy the DSA staff)
access_key = 'Put your Access Secret Key here'

#Security group name
Sec_group_name_west= "SG_your_pawprint_West_Oregon"

In [ ]:
#Import AWS' Python Based DEVOPS tools
import boto3
from botocore.exceptions import ClientError

#Import System Tools
import collections
import json
import os
import datetime
import pandas
import time
import getpass
from subprocess import call

#Set the username from system
system_user_name=getpass.getuser()

# client interface.
# Estabilish Credentials/Session
ec2 = boto3.client(
    'ec2', 
    region_name=region,
    aws_access_key_id=access_id,
    aws_secret_access_key=access_key
)

Create a security group to ve used while launching instances. 
The inbound rules of the security group should allow SSH requests so we can get into the instances.
All all TCP traffic and the ICMP requests to do ping requests. 

### Test 1:

In [ ]:
sg_group = ec2.create_security_group(
    Description='security grp for docker',
    GroupName=Sec_group_name_west   # We have set this variable above
)
Sec_group=sg_group["GroupId"]     # Sec_group should have the new security group ID.

### Create a Security group for the Oregon center instances 

Ping operates by sending Internet Control Message Protocol (ICMP/ICMP6) Echo Request packets to the target host and waiting for an ICMP Echo Reply. The echo request ("ping") is an ICMP/ICMP6 message. The echo reply is an ICMP message generated in response to an echo request. For ICMP protocol, we dont specify ports, rather we specify message types. To allow ping requests, add the ICMP protocol type and specify 8 (echo request) for the ICMP type and either 0 or -1 (all) for the ICMP code.

For example,

`"SecurityGroupIngress" : [ 
      { "IpProtocol" : "tcp", "FromPort" : "22", "ToPort" : "22", "CidrIp" : "10.0.0.0/24" },
      { "IpProtocol" : "icmp", "FromPort" : "8", "ToPort" : "-1", "CidrIp" : "10.0.0.0/24" }
    ]`
    


In below cell, we are setting security inbound rules for the security group "sg_group" we created above. 

### Note:

Please ckeck for your public IP address in google. Just type "my ip address" in search box. Replace the IP address (xxx.xxx.xxx.xxx/32)in below code cell for ICMP protocol with your ip address. 

In [ ]:
try:
    sec_rule="Custom TCP Rule"    
    ec2.authorize_security_group_ingress(GroupId=Sec_group,IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=22,ToPort=22)
    print(sec_rule+" added")
except:
    print(sec_rule+" already added")
try:
    sec_rule="ALL TCP"
    ec2.authorize_security_group_ingress(GroupId=Sec_group,IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=0,ToPort=65535)
    print(sec_rule+" added")
except:
    print(sec_rule+" already added")
try:
    sec_rule="ALL ICMP"
    ec2.authorize_security_group_ingress(GroupId=Sec_group,IpProtocol="ICMP",CidrIp="0.0.0.0/0",FromPort=-1,ToPort=-1)
    print(sec_rule+" added")
except:
    print(sec_rule+" already added")
try:
    sec_rule="Custom ICMP Rule"
    ec2.authorize_security_group_ingress(GroupId=Sec_group,IpProtocol="ICMP",CidrIp="xxx.xxx.xxx.xxx/32",FromPort=0,ToPort=-1)
    print(sec_rule+" added")
except:
    print(sec_rule+" already added")  

Get the Id of the security group using describe_security_groups() function. The response returns detials of all security groups. Filter the results by specifying the security group name and extract only the required piece of information using key-value pairs. 

In [ ]:
desc_sec_groups = ec2.describe_security_groups(
    GroupNames=[
        Sec_group_name_west,
    ]
)
sg_grp_id1 = desc_sec_groups["SecurityGroups"][0]["GroupId"]
sg_grp_id1

Generate keypair. 

In [ ]:
ec2_pem_file1=time.strftime("EMR-%d%m%Y%H%M%S-"+system_user_name)
ec2_key1=ec2.create_key_pair(KeyName=ec2_pem_file1)

#Don't do this unless you have a good reason
#print(emr_key['KeyMaterial'])

os.system("echo \""+ec2_key1['KeyMaterial']+"\" > "+ec2_pem_file1+".pem")
os.chmod(ec2_pem_file1+".pem",400)

print("KeyName         : "+ec2_key1['KeyName']+"\nKey Fingerprint : "+ec2_key1['KeyFingerprint'])

### Launch instances in Oregon center

Use the security group Id identified above to launch 2 instances in Oregon data center. Use the Keypair we just generated. We need 2 instances in Oregon center. 



<!-- 
    The AWS machine Id is picked from AWS console.
    <img src="../images/ami.PNG">
-->

In [ ]:
# Creating 2 Instances in Oregon data center 
instances = ec2.run_instances(
    ImageId=ami_image,
    MinCount=2, 
    MaxCount=2,
    KeyName=ec2_pem_file1,
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                        {   'Key': 'Name',
                            'Value': 'pingserver_Oregon'
                        }
                    ]
        }
    ],
    InstanceType="t2.micro",
    SecurityGroupIds=[
        sg_grp_id1
    ]
)

# <span style="background:pink">Wait until the instances are launched before running the following cell</span>
Once the instances are launched, get their tags information, their public IP addresses, avaialbility region and also the public DNS name. We need them in later cells to SSH into them. We used filters to list only t2.micro instances as we know the AMI we picked launches t2.micro instances. The filter, "instance-state-code" with value 16 means the instances that are currently running. So we are going to see all running t2.micro instances at the moment along with their Availability region, Public IP address and public DNS name. 

In [ ]:
# Get the DNS address, tags and public IP address of instances. We need the DNS address next to SSH into it
Oregon_ins_ids=[]
for region in ec2.describe_regions()['Regions']:
    
    idesc = ec2.describe_instances(Filters=[
        {
            'Name': 'instance-state-code',
            'Values': ['16',
            ]
        },
        {
            'Name': 'instance-type',
            'Values': ['t2.micro',
            ]
        },
        {
            'Name':'tag:Name',
            'Values':['pingserver_Dublin','pingserver_Oregon']    # Filter instances which have tag pingserver in them.
        }
    ],)

#print(idesc)
for r in idesc['Reservations']:
        for i in r['Instances']:
            if(i['Tags'][0]['Value'] in ["pingserver_Oregon"]):
                Oregon_ins_ids.append(i["InstanceId"])

            print('Region: ',i["Placement"]["AvailabilityZone"],'\n',
                  'Public Dns Name: ', i['PublicDnsName'],'\n','Tags: ', 
                  i["Tags"],'\n','Public Ip Address: ', i["PublicIpAddress"], '\n',
                  "Instance id: ",i["InstanceId"])
            print('*'*40)

In [ ]:
Oregon_ins_ids

The code in below cell invokes a command shell and shows the ping requests. 

[Reference for invoke_shell() used in below cell](http://docs.paramiko.org/en/2.2/api/channel.html#paramiko.channel.Channel.invoke_shell)


## Note: 

Change below listed variables in the code cell below. Use the details we printed above. The **"host DNS"** would be the first instance public DNS name which among ones with tag "pingserver_Oregon". filename is the keypair used to launch the instance. cmd is the ping command with public ip address of second EC2 instance with tag "pingserver_Oregon".


* host_DNS = $<First\ instances\ public\ DNS\ name>$
* filename = $<keypair\ used\ to\ launch\ instances>$
* cmd = $<Public\ IP\ address\ of\ the\ second\ instance>$



In [ ]:
host_DNS = "ec2-35-162-55-98.us-west-2.compute.amazonaws.com"
username='ec2-user'
filename=ec2_pem_file1+".pem"

cmd = "ping 34.214.172.61"

In [ ]:
# ping one instance in Oregon center to another instance.

import paramiko
import sys
import time

class sampleParamiko:
    ssh = ""
    def __init__(self, host_DNS, uname, keyfile):
        try:
            self.ssh = paramiko.SSHClient()
            self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            self.ssh.connect(host_DNS, username=uname, key_filename=keyfile)
            #print "In init function"
        except (paramiko.BadHostKeyException, paramiko.AuthenticationException, paramiko.SSHException) as e:
            print(str(e))
            sys.exit(-1)

    def executeCmd(self,cmd):
        try:
            channel = self.ssh.invoke_shell()
            timeout = 60 # timeout is in seconds
            channel.settimeout(timeout)
            newline        = '\r'
            line_buffer    = ''
            channel_buffer = ''
            channel.send(cmd + ' ; exit ' + newline)
            
            endTime = datetime.datetime.now() + datetime.timedelta(seconds=5)
    
            while True:
                if datetime.datetime.now() >= endTime:
                    break
                channel_buffer = channel.recv(1).decode('UTF-8')
                if len(channel_buffer) == 0:
                    break 
                channel_buffer  = channel_buffer.replace('\r', '')
                if channel_buffer != '\n':
                    line_buffer += channel_buffer
                else:
                    print(line_buffer)
                    line_buffer   = ''

        except paramiko.SSHException as e:
            print(str(e))
            sys.exit(-1)
            
conn_obj = sampleParamiko(host_DNS, username, filename)
conn_obj.executeCmd(cmd)

So it took less than one milli second to ping from one instance to another one both located in Oregon center.  

### Test 2:

Repeat this experiment but pinging the instance in Ireland center from one of the instances in Oregon center.


Create a new ec2 resource object for Ireland center as we have to create a security for that center. 

In [ ]:
################################### SET THE FOLLOWING PARAMETERS ###################################################
#Set the AWS Region
region = 'eu-west-1'


dublin_ami_image = 'ami-0ee415e1b8b71305f'

#Set the AWS Access ID (Given to you buy the DSA staff)
#Probably, no need to set this since it was set above
#access_id = 'Put your Access key ID here'


#Set the AWS Access Key (Given to you buy the DSA staff)
#Probably, no need to set this since it was set above
#access_key = 'Put your Access Secret here'

#Security group name
Sec_group_name_Dublin= "SG_your_pawprint_Dublin"

In [ ]:
#Import AWS' Python Based DEVOPS tools
import boto3
from botocore.exceptions import ClientError

#Import System Tools
import collections
import json
import os
import datetime
import pandas
import time
import getpass
from subprocess import call

#Set the username from system
system_user_name=getpass.getuser()

# client interface.
# Estabilish Credentials/Session
ec2_Dublin = boto3.client(
    'ec2', 
    region_name=region,
    aws_access_key_id=access_id,
    aws_secret_access_key=access_key
)

In [ ]:
sg_group2 = ec2_Dublin.create_security_group(
    Description='This security group is for accessing instance in Ireland',
    GroupName=Sec_group_name_Dublin   # We have set this variable above
)
Sec_group2=sg_group2["GroupId"]     # Sec_group should have the new security group ID.

Update the security rules just like we did for Oregon security group

In [ ]:
try:
    sec_rule="Custom TCP Rule"    
    ec2_Dublin.authorize_security_group_ingress(GroupId=Sec_group2,IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=22,ToPort=22)
    print(sec_rule+" added")
except:
    print(sec_rule+" already added")
try:
    sec_rule="ALL TCP"
    ec2_Dublin.authorize_security_group_ingress(GroupId=Sec_group2,IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=0,ToPort=65535)
    print(sec_rule+" added")
except:
    print(sec_rule+" already added")
try:
    sec_rule="ALL ICMP"
    ec2_Dublin.authorize_security_group_ingress(GroupId=Sec_group2,IpProtocol="ICMP",CidrIp="0.0.0.0/0",FromPort=-1,ToPort=-1)
    print(sec_rule+" added")
except:
    print(sec_rule+" already added")
try:
    sec_rule="Custom ICMP Rule"
    ec2_Dublin.authorize_security_group_ingress(GroupId=Sec_group2,IpProtocol="ICMP",CidrIp="xxx.xxx.xxx.xxx/32",FromPort=0,ToPort=-1)
    print(sec_rule+" added")
except:
    print(sec_rule+" already added")  

Getting the security group Id by running describe_security_groups() method. 

In [ ]:
desc_sec_groups = ec2_Dublin.describe_security_groups(
    GroupNames=[
        Sec_group_name_Dublin,
    ]
)
sg_grp_id2 = desc_sec_groups["SecurityGroups"][0]["GroupId"]
sg_grp_id2

Finally create a keypair for Ireland region as we are about to create an instance.

In [ ]:
ec2_pem_file2=time.strftime("EMR-dublin-%d%m%Y%H%M%S-"+system_user_name)
ec2_key2=ec2_Dublin.create_key_pair(KeyName=ec2_pem_file2)

#Don't do this unless you have a good reason
#print(emr_key['KeyMaterial'])

os.system("echo \""+ec2_key2['KeyMaterial']+"\" > "+ec2_pem_file2+".pem")
os.chmod(ec2_pem_file2+".pem",400)

print("KeyName         : "+ec2_key2['KeyName']+"\nKey Fingerprint : "+ec2_key2['KeyFingerprint'])

Run one instance using the keypair and security group created above for Ireland center. 

In [ ]:
# Creating 1 Instances in Ireland data center 
instances = ec2_Dublin.run_instances(
    ImageId=dublin_ami_image,
    MinCount=1, 
    MaxCount=1,
    KeyName=ec2_pem_file2,
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                        {   'Key': 'Name',
                            'Value': 'pingserver_Dublin'
                        }
                    ]
        }
    ],
    InstanceType="t2.micro",
    SecurityGroupIds=[
        sg_grp_id2,
    ]
)


# <span style="background:pink">Wait until the instance is launched before running the following cell</span>


In [ ]:
# Get the DNS address, tags and public IP address of all instances. We need the DNS addresses in next cell to SSH into them

# Empty list to capture the instance ids of all instances we just created. Instance ids are captured
# to delete the instances once we are done with the experiment.
Dublin_ins_ids=[]

for region in ec2_Dublin.describe_regions()['Regions']:
    
    idesc = ec2_Dublin.describe_instances(Filters=[
        {
            'Name': 'instance-state-code',    # Filter instances who are currently running
            'Values': ['16',
            ]
        },
        {
            'Name': 'instance-type',         # Filter instances whose type is "t2.micro"
            'Values': ['t2.micro',
            ]
        },
        {
            'Name':'tag:Name',
            'Values':['pingserver_Dublin','pingserver_Oregon']    # Filter instances which have tag pingserver in them.
        }
    ],)
#     print(idesc)
    
for r in idesc['Reservations']:
        for i in r['Instances']:
            if(i["Tags"][0]['Value'] in ["pingserver_Dublin"]):
                Dublin_ins_ids.append(i["InstanceId"])

            print('Region: ',i["Placement"]["AvailabilityZone"],'\n',
              'Public Dns Name: ', i['PublicDnsName'],'\n','Tags: ', 
              i["Tags"],'\n','Public Ip Address: ', i["PublicIpAddress"], '\n',
              "Instance id: ",i["InstanceId"])
            print('*'*40)

The instance ids are stored in in_ids variable. 

In [ ]:
print(Oregon_ins_ids)
print(Dublin_ins_ids)

In [ ]:
dublin_host_DNS = "ec2-52-50-224-62.eu-west-1.compute.amazonaws.com"     # Dublin instance Public DNS address
username='ec2-user'
filename=ec2_pem_file2+".pem"

# You can probably just leave this from above,
# but it should be one of the Oregon instance's public IP addresses
# cmd = "ping 54.70.234.116"

In [ ]:
# ping from (instance in Ireland) to (instance in Oregon)
# Pinging a EC2 instance in Oregon from Dublin instance


import paramiko
import sys
import time

class sampleParamiko:
    ssh = ""
    def __init__(self, host_DNS, uname, keyfile):
        try:
            self.ssh = paramiko.SSHClient()
            self.ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
            self.ssh.connect(host_DNS, username=uname, key_filename=keyfile)
            #print "In init function"
        except (paramiko.BadHostKeyException, paramiko.AuthenticationException, paramiko.SSHException) as e:
            print(str(e))
            sys.exit(-1)

    def executeCmd(self,cmd):
        try:
            channel = self.ssh.invoke_shell()
            timeout = 60 # timeout is in seconds
            channel.settimeout(timeout)
            newline        = '\r'
            line_buffer    = ''
            channel_buffer = ''
            channel.send(cmd + ' ; exit ' + newline)
            
            endTime = datetime.datetime.now() + datetime.timedelta(seconds=5)
    
            while True:
                if datetime.datetime.now() >= endTime:
                    break
                channel_buffer = channel.recv(1).decode('UTF-8')
                if len(channel_buffer) == 0:
                    break 
                channel_buffer  = channel_buffer.replace('\r', '')
                if channel_buffer != '\n':
                    line_buffer += channel_buffer
                else:
                    print(line_buffer)
                    line_buffer   = ''

        except paramiko.SSHException as e:
            print(str(e))
            sys.exit(-1)
            
conn_obj = sampleParamiko(dublin_host_DNS, username, filename)
conn_obj.executeCmd(cmd)

There it is, It is taking a 100 times longer to complete a ping request between an instance in Oregon and one in Ireland. This proves the point made earlier in this notebook that geographic location of physical servers have impact on communication overhead, speed, service quality and user experience etc. 

# Delete the Keypairs created

In [ ]:
# Delete SSH Keypair

try:
    os.remove(ec2_pem_file1+'.pem')
    print('Local Key Deleted')

    os.remove(ec2_pem_file2+'.pem')
    print('Local Key Deleted')

except:
    print('Local Key Not Found')
    
delete_keys_response1 = ec2.delete_key_pair(KeyName=ec2_pem_file1)
print('\nAWS Metadata: ')
print('http Status Code : '+str(delete_keys_response1['ResponseMetadata']['HTTPStatusCode']))
print('Request ID       : '+delete_keys_response1['ResponseMetadata']['RequestId'])
print('Retries          : '+str(delete_keys_response1['ResponseMetadata']['RetryAttempts']))

delete_keys_response2 = ec2_Dublin.delete_key_pair(KeyName=ec2_pem_file2)
print('\nAWS Metadata: ')
print('http Status Code : '+str(delete_keys_response2['ResponseMetadata']['HTTPStatusCode']))
print('Request ID       : '+delete_keys_response2['ResponseMetadata']['RequestId'])
print('Retries          : '+str(delete_keys_response2['ResponseMetadata']['RetryAttempts']))

# Delete the instances created

In [ ]:
import numpy as np

for i in np.arange(0,len(Oregon_ins_ids)):
    out = ec2.terminate_instances(
    InstanceIds=[Oregon_ins_ids[i]])
    print(out)
    print('*'*40)
    
for i in np.arange(0,len(Dublin_ins_ids)):
    out = ec2_Dublin.terminate_instances(
    InstanceIds=[Dublin_ins_ids[i]])
    print(out)
    print('*'*40)

# <span style="background:pink">Wait for the instances to shut down. You cannot delete a security group when an instance is using it. </span>



# Delete the Security Group

In [ ]:
del_Org_SG_response = ec2.delete_security_group(
    GroupId=sg_grp_id1
)
print(del_Org_SG_response)

del_Dub_SG_response = ec2_Dublin.delete_security_group(
    GroupId=sg_grp_id2
)
print(del_Dub_SG_response)

## Using terminal to ping the instances


Its easy to the same thing as above using terminal. 
We have all 3 three EC2 instances(two in Oregon center and one in Dublin center) up and running. 
SSH into them and ping one another. 
In below screen shot we SSHed into Oregon instance using KeyPair `EC2KeyPairDublin.pem` 
and pinged one server in Oregon region. It took around 140 ms for a response in ping request.



<img src="../images/dublin_to_Oregon.PNG">



-----


In below screen shot we SSHed into one of the instance in Oregon using KeyPair `EC2KeyPair1.pem` 
and pinged other server in Oregon region. 
It took around 0.5 to 0.7 ms to get a response in ping request. 
Since the time taken is in milli seconds we cannot feel any difference in time taken 
to get a response in ping request but when you are dealing with Giga bytes of data then it makes a huge difference. 
You might transfer files, intermediate processing results, 
pipeline processing between instances etc. 
So its important to choose instances wisely based on their geographic location.   


<img src="../images/Oregon_to_Oregon.PNG">

# Save your notebook, then `File > Close and Halt`